# Hespress Sentiment Analysis

**Réalisé par:**
>Wassima MANSSOUR <br>
>Sokaina NADI

# **1. Scraping**

## Installation des librairies nécessaires

In [ ]:
!pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 211 kB 12.8 MB/s 
     |████████████████████████████████| 7.4 MB 61.9 MB/s 
     |████████████████████████████████| 81 kB 11.0 MB/s 
     |████████████████████████████████| 93 kB 3.4 MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=c476e2687c4482a14743ea5c9a05dc25f4582c0f31df050ddb46ba4e5df346f5
  Stored in directory: /root/.cache/pip/wheels/99/74/83/8fac1c8d9c648cfabebbbffe97a889f6624817f3aa0bbe6c09
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3354 sha256=17559791907e7bfc084e64b1e8a741acf969ec184c10cd320b7a24d03fb1dc2a
  Stored in directory: /root/.cache/pip/wheels/b6/09/68/a9f15498ac02c23dde29f18745bc6a6f574ba4ab41861a3575
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398405 sha256=14ce0ac18ca3ec961f2181b3a694340dbe44df5b7cc12ac

In [ ]:
from newspaper import Article
import newspaper
import sqlite3
import requests
import re
from datetime import date, datetime
import bs4
from time import sleep
import os
import pandas as pd
import numpy as np


## Extraction des liens des articles

In [ ]:
url = 'https://www.hespress.com/tag/headlines'

hespress_paper = newspaper.build(url,language='ar')

CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.hespress.com/feeds


In [ ]:
print(hespress_paper.size())

220


In [ ]:
articles_links = []

for article in hespress_paper.articles:
  articles_links.append(article.url)

In [ ]:
df = pd.DataFrame(articles_links,columns=['Article Link'])

In [ ]:
df

,Article Link
0,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...
1,https://www.hespress.com/%d8%a7%d9%84%d8%b1%d8...
2,https://www.hespress.com/%d9%85%d9%8a%d8%b1%d8...
3,https://www.hespress.com/%d8%b1%d8%b5%d9%8a%d9...
4,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...
...,...
215,https://en.hespress.com/42299-pcr-tests-no-lon...
216,https://en.hespress.com/31107-morocco-suspends...
217,https://en.hespress.com/54429-regragui-has-not...
218,https://en.hespress.com/55009-arab-amazigh-bot...


## Extraction des titres et des articles

In [ ]:
article_text=[]
article_titles=[]

for link in articles_links:
  artcl = Article(link, language="ar")
  artcl.download()
  artcl.parse()
  t=artcl.text
  article_text.append(t)
  t2=artcl.title
  article_titles.append(t2)

In [ ]:
df = pd.DataFrame(list(zip(articles_links,article_titles,article_text)),columns=['Article Link','Article_title','Article'])

In [ ]:
df

,Article Link,Article_title,Article
0,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تخصص 28 مليار درهم لمواجهة التضخم وتدا...,كشف فوزي لقجع، الوزير المنتدب لدى وزير الاقتصا...
1,https://www.hespress.com/%d8%a7%d9%84%d8%b1%d8...,الرئيس التونسي قيس سعيّد يتجاهل المقاطعة الواس...,قلّل الرئيس التونسي قيس سعيّد، الأربعاء، من أه...
2,https://www.hespress.com/%d9%85%d9%8a%d8%b1%d8...,ميراوي يعترف بصعوبة تعويض الأساتذة المقبلين عل...,قال عبد اللطيف ميراوي، وزير التعليم العالي وال...
3,https://www.hespress.com/%d8%b1%d8%b5%d9%8a%d9...,رصيف الصحافة: أطباء يحذرون من اتساع الممارسة ا...,قراءة مواد بعض الجرائد الخاصة بيوم الخميس نسته...
4,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تنجح في إقناع مهندسي الملاحة الجوية ال...,تراجع مهندسو وأطر سلامة الملاحة الجوية بمطارات...
...,...,...,...
215,https://en.hespress.com/42299-pcr-tests-no-lon...,PCR tests no longer required to enter Morocco,
216,https://en.hespress.com/31107-morocco-suspends...,Morocco suspends all passenger flights to the ...,
217,https://en.hespress.com/54429-regragui-has-not...,"Regragui has 'nothing to lose,' says Atlas lio...",
218,https://en.hespress.com/55009-arab-amazigh-bot...,"Arab, Amazigh, both and neither: Moroccans gra...",


### Elimination des NaN et des articles indesirables

In [ ]:
df['Article'].replace('', np.nan, inplace=True)
# Sélection des lignes qui ne contiennent pas de NaN dans la colonne "ma_colonne"
df.dropna(subset=['Article'], inplace=True)

In [ ]:
df

,Primary_Key,Article Link,Article_title,Article
0,0,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تخصص 28 مليار درهم لمواجهة التضخم وتدا...,كشف فوزي لقجع، الوزير المنتدب لدى وزير الاقتصا...
1,1,https://www.hespress.com/%d8%a7%d9%84%d8%b1%d8...,الرئيس التونسي قيس سعيّد يتجاهل المقاطعة الواس...,قلّل الرئيس التونسي قيس سعيّد، الأربعاء، من أه...
2,2,https://www.hespress.com/%d9%85%d9%8a%d8%b1%d8...,ميراوي يعترف بصعوبة تعويض الأساتذة المقبلين عل...,قال عبد اللطيف ميراوي، وزير التعليم العالي وال...
3,3,https://www.hespress.com/%d8%b1%d8%b5%d9%8a%d9...,رصيف الصحافة: أطباء يحذرون من اتساع الممارسة ا...,قراءة مواد بعض الجرائد الخاصة بيوم الخميس نسته...
4,4,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تنجح في إقناع مهندسي الملاحة الجوية ال...,تراجع مهندسو وأطر سلامة الملاحة الجوية بمطارات...
...,...,...,...,...
158,154,https://www.hespress.com/%d8%af%d8%b1%d9%88%d8...,"دروس وعبر من ""مونديال"" قطر!",انتهت كما هو معلوم منافسات كأس العالم في نسخته...
159,155,https://www.hespress.com/%d8%ad%d9%8a%d9%86-%d...,حين عاد الأبطال الصناديد إلى أرض الوطن,بعد ملحمة الدوحة، عاد أبناء الوطن الى حضن الوط...
160,156,https://www.hespress.com/%d8%a7%d9%84%d8%aa%d8...,التعليم.. إصلاح قد لا يأتي,المتأمل لكرونولوجيا ما يسمى “إصلاح التعليم” با...
161,157,https://www.hespress.com/%d8%a7%d9%84%d9%88%d9...,الوكالات الجهوية للتعمير في خدمة الاسثثمار,راكمت الوكالات الحضرية، منذ إحداثها خلال عقد ا...


In [ ]:
pd.set_option('max_rows', None)
df

,Primary_Key,Article Link,Article_title,Article
0,0,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تخصص 28 مليار درهم لمواجهة التضخم وتدا...,كشف فوزي لقجع، الوزير المنتدب لدى وزير الاقتصا...
1,1,https://www.hespress.com/%d8%a7%d9%84%d8%b1%d8...,الرئيس التونسي قيس سعيّد يتجاهل المقاطعة الواس...,قلّل الرئيس التونسي قيس سعيّد، الأربعاء، من أه...
2,2,https://www.hespress.com/%d9%85%d9%8a%d8%b1%d8...,ميراوي يعترف بصعوبة تعويض الأساتذة المقبلين عل...,قال عبد اللطيف ميراوي، وزير التعليم العالي وال...
3,3,https://www.hespress.com/%d8%b1%d8%b5%d9%8a%d9...,رصيف الصحافة: أطباء يحذرون من اتساع الممارسة ا...,قراءة مواد بعض الجرائد الخاصة بيوم الخميس نسته...
4,4,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تنجح في إقناع مهندسي الملاحة الجوية ال...,تراجع مهندسو وأطر سلامة الملاحة الجوية بمطارات...
5,5,https://www.hespress.com/%d8%a7%d9%84%d9%84%d8...,"""اللجنة العلمية"" تحذر المغاربة من تزاوج ""فيروس...",يعاني عدد كبير من المواطنين، خلال الفترة الحال...
6,6,https://www.hespress.com/%d8%a7%d8%b3%d8%aa%d8...,"استبعاد ""دونور"" من ""الموندياليتو"" يجدد مطالب ك...",أثار استبعاد المركب الرياضي محمد الخامس بالدار...
7,7,https://www.hespress.com/%d9%88%d9%87%d8%a8%d9...,وهبي يوضح بشأن محاسبة المتماطلين في الأحكام وي...,عاد عبد اللطيف وهبي، وزير العدل، للحديث مجددا ...
8,8,https://www.hespress.com/%d8%aa%d9%82%d8%b1%d9...,"تقرير حقوقي يرصد انتشارا ""مخيفا"" للهدر المدرسي...",كشف تقرير حقوقي أنجزته “مؤسسة يطو لإيواء وإعاد...
9,9,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,"الحكومة و""إيركام"" يشرعان في تفعيل إدماج الأماز...",تتجه الحكومة المغربية، فعليا، إلى تفعيل الطابع...


In [ ]:
df.to_csv(r'article_info.csv', index=False, header=True)

In [ ]:
df['Article'][87]

'النشرة الإخبارية\n\nاشترك الآن في النشرة البريدية لجريدة هسبريس، لتصلك آخر الأخبار يوميا'

In [ ]:

print(df[df["Article"].str.contains("شترك الآن")])


     Primary_Key                                       Article Link  \
87            87  https://www.hespress.com/%d8%aa%d8%aa%d9%88%d9...   
118          114  https://www.hespress.com/%d8%a7%d9%84%d9%85%d8...   
119          115  https://www.hespress.com/%d8%a5%d8%af%d9%85%d8...   
120          116  https://www.hespress.com/%d8%a7%d9%84%d8%aa%d9...   
121          117  https://www.hespress.com/%d9%85%d9%82%d8%a7%d9...   
130          126  https://www.hespress.com/%d9%82%d8%a7%d9%86%d9...   
131          127  https://www.hespress.com/%d8%a7%d9%86%d8%ad%d8...   
132          128  https://www.hespress.com/%d8%aa%d8%b7%d8%a8%d9...   
133          129  https://www.hespress.com/%d8%b9%d9%85%d8%a7%d9...   
134          130  https://www.hespress.com/%d8%a5%d8%b4%d8%a7%d8...   

                                 Article_title  \
87                    تتويج جديد يثير فرح آسفي   
118     المختصر | انطلاق إحصاء الخدمة العسكرية   
119       إدماج الأمازيغية في التشريع والتنظيم   
120        التكفل 

In [ ]:
df_articles = df[~df["Article"].str.contains("شترك الآن")]


In [ ]:
len(df_articles)

83

In [ ]:
print(df_articles[df_articles["Article"].str.contains("شترك الآن")])

Empty DataFrame
Columns: [Primary_Key, Article Link, Article_title, Article]
Index: []


### Ajouter une colonne de clé primaire

In [ ]:
values = df_articles['Article_title'].unique()
values = pd.Series(np.arange(len(values)), values)
df_articles['Primary_Key'] = df_articles['Article_title'].apply(values.get)

<ipython-input-41-5999dd8d660a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_articles['Primary_Key'] = df_articles['Article_title'].apply(values.get)


In [ ]:
# Définition de l'ordre souhaité des colonnes
col_order = ['Primary_Key', 'Article Link', 'Article_title','Article']

# Réindexation du DataFrame avec l'ordre des colonnes souhaité
df_articles = df_articles.reindex(columns=col_order)

In [ ]:
df_articles.reset_index(inplace=True)

In [ ]:
#df_articles = df_articles.drop(labels='index', axis=1)

In [ ]:
df_articles

,Primary_Key,Article Link,Article_title,Article
0,0,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تخصص 28 مليار درهم لمواجهة التضخم وتدا...,كشف فوزي لقجع، الوزير المنتدب لدى وزير الاقتصا...
1,1,https://www.hespress.com/%d8%a7%d9%84%d8%b1%d8...,الرئيس التونسي قيس سعيّد يتجاهل المقاطعة الواس...,قلّل الرئيس التونسي قيس سعيّد، الأربعاء، من أه...
2,2,https://www.hespress.com/%d9%85%d9%8a%d8%b1%d8...,ميراوي يعترف بصعوبة تعويض الأساتذة المقبلين عل...,قال عبد اللطيف ميراوي، وزير التعليم العالي وال...
3,3,https://www.hespress.com/%d8%b1%d8%b5%d9%8a%d9...,رصيف الصحافة: أطباء يحذرون من اتساع الممارسة ا...,قراءة مواد بعض الجرائد الخاصة بيوم الخميس نسته...
4,4,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تنجح في إقناع مهندسي الملاحة الجوية ال...,تراجع مهندسو وأطر سلامة الملاحة الجوية بمطارات...
5,5,https://www.hespress.com/%d8%a7%d9%84%d9%84%d8...,"""اللجنة العلمية"" تحذر المغاربة من تزاوج ""فيروس...",يعاني عدد كبير من المواطنين، خلال الفترة الحال...
6,6,https://www.hespress.com/%d8%a7%d8%b3%d8%aa%d8...,"استبعاد ""دونور"" من ""الموندياليتو"" يجدد مطالب ك...",أثار استبعاد المركب الرياضي محمد الخامس بالدار...
7,7,https://www.hespress.com/%d9%88%d9%87%d8%a8%d9...,وهبي يوضح بشأن محاسبة المتماطلين في الأحكام وي...,عاد عبد اللطيف وهبي، وزير العدل، للحديث مجددا ...
8,8,https://www.hespress.com/%d8%aa%d9%82%d8%b1%d9...,"تقرير حقوقي يرصد انتشارا ""مخيفا"" للهدر المدرسي...",كشف تقرير حقوقي أنجزته “مؤسسة يطو لإيواء وإعاد...
9,9,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,"الحكومة و""إيركام"" يشرعان في تفعيل إدماج الأماز...",تتجه الحكومة المغربية، فعليا، إلى تفعيل الطابع...


## Extraction des commentaires

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 14.3 MB/s 
     |████████████████████████████████| 140 kB 79.2 MB/s 
     |████████████████████████████████| 384 kB 75.8 MB/s 
     |████████████████████████████████| 58 kB 8.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,235 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download.nvidia.co

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-div-shm-usage")
driver = webdriver.Chrome("chromedriver",options=chrome_options)

In [ ]:
info=[]
for i in range (0,len(df_articles)):
  comm=[]
  autor=[]
  reactions=[]
  Foreign_Key=df_articles['Primary_Key'][i]
  url=df_articles['Article Link'][i]
  driver.get(url)
  comments = driver.find_elements(By.CLASS_NAME,"comment-text")
  author = driver.find_elements(By.CLASS_NAME,"fn")
  reaction = driver.find_elements(By.CLASS_NAME,"comment-recat-number")
  for i in reaction:
    reactions.append(i.text)
  for i in comments:
    comm.append(i.text)
  for j in author:
    autor.append(j.text)
  for i in range(len(comm)):
          pageinfo={}
          pageinfo['Foreign_Key']=Foreign_Key
          pageinfo['auteurComm']=autor[i]
          pageinfo['Comm']=comm[i]
          pageinfo['Number of reaction']=reactions[i]
          info.append(pageinfo)


In [ ]:
import pandas as pd
df_comm=pd.DataFrame(info,columns=["Foreign_Key",'auteurComm','Comm','Number of reaction'])
df_comm

,Foreign_Key,auteurComm,Comm,Number of reaction
0,0,Stop,للكرة وللحباب ولصحاب لملايير كاينة وملي الشعب ...,4
1,0,redos,تبارك الله اللهم بارك هاذ لقجع فينما مشيتي تلق...,2
2,0,Diego,هل يمكن اتخاد قرارات مثل التي اتخذتها اسبانيا ...,3
3,1,محمد الطربلوسي,كذلك فعل المخلوع زين العابدين بن علي،ناداه الش...,9
4,1,ولد حميدو,من حقه ان يتجاهل فهل الشعب هو من أعطاه 300 ملي...,5
5,1,mda,منذ الثورة التي أطاحت بالدكتاتورية في العام201...,6
6,1,اضمين عبدالله,قيس سعيد فوق القانون ويجهل ما قاله الشاعر التو...,2
7,1,عزيز,رجل فاشل لا يستحق أن يكون في الرئاسة. مهووس با...,2
8,1,bifnza,هذا دليل على انه يثق في نفسه وهذه الثقة تدل عل...,0
9,1,بوعلام,الرئيس السابق منصف المرزوقي: “تونس اليوم مثل ح...,2


## Cleaning Data


In [ ]:
#Remove punctuations, links, mentions and \r\n new line characters
import string
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text=re.sub("\u200e\u200e","",text)
    text = re.sub("\u200e\u200e"," ",text)
    text = re.sub("\u200f"," ",text)
    text = re.sub("\u200e"," ",text)
    text = re.sub("\u202c"," ",text)
    text = re.sub("\u202a"," ",text)
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    #text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

In [ ]:
#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

In [ ]:
#Clean emojis from text
import re
txt = 'He is so happy 🙂🙂' # <-- grepper doesn't support emoji, so just assume it has
def strip_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
print(strip_emoji(txt))

He is so happy 


In [ ]:
#cleaning comments
texts_new = []
for t in df_comm.Comm:
    texts_new.append(remove_mult_spaces(filter_chars(strip_all_entities(strip_emoji(t)))))

In [ ]:
#cleaning articles
articles= []

for t in df_articles.Article:
    articles.append(remove_mult_spaces(filter_chars(strip_all_entities(strip_emoji(t)))))

In [ ]:
#cleaning title of articles
titres= []

for t in df_articles.Article_title:
    titres.append(remove_mult_spaces(filter_chars(strip_all_entities(strip_emoji(t)))))

In [ ]:
df_articles.head()

,Primary_Key,Article Link,Article_title,Article
0,0,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تخصص 28 مليار درهم لمواجهة التضخم وتدا...,كشف فوزي لقجع، الوزير المنتدب لدى وزير الاقتصا...
1,1,https://www.hespress.com/%d8%a7%d9%84%d8%b1%d8...,الرئيس التونسي قيس سعيّد يتجاهل المقاطعة الواس...,قلّل الرئيس التونسي قيس سعيّد، الأربعاء، من أه...
2,2,https://www.hespress.com/%d9%85%d9%8a%d8%b1%d8...,ميراوي يعترف بصعوبة تعويض الأساتذة المقبلين عل...,قال عبد اللطيف ميراوي، وزير التعليم العالي وال...
3,3,https://www.hespress.com/%d8%b1%d8%b5%d9%8a%d9...,رصيف الصحافة: أطباء يحذرون من اتساع الممارسة ا...,قراءة مواد بعض الجرائد الخاصة بيوم الخميس نسته...
4,4,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تنجح في إقناع مهندسي الملاحة الجوية ال...,تراجع مهندسو وأطر سلامة الملاحة الجوية بمطارات...


In [ ]:
df_articles['Clean_article'] = articles

In [ ]:
df_comm['Clean_comment'] = texts_new

In [ ]:
df_articles['Clean_Title']=titres

In [ ]:
df_articles['Article'][0]

'كشف فوزي لقجع، الوزير المنتدب لدى وزير الاقتصاد والمالية المكلف بالميزانية، أن الحكومة لجأت، خلال السنة الحالية، إلى فتح اعتمادات إضافية بمرسوم، بلغت 28 مليار درهم لمواجهة تداعيات تقلب أسعار بعض المواد الأولية وارتفاع معدل التضخم بفعل الحرب في أوكرانيا.\n\nويأتي هذا الإجراء في إطار ما يتمتع به الجهاز التنفيذي من صلاحيات لإجراء تغييرات على مستوى نفقات ميزانية الدولة المصادق عليها أثناء تنفيذ قانون المالية، وفق شروط وسياق يؤطرها القانون التنظيمي لقانون المالية؛ وذلك في إطار تمكينه من الوسائل الضرورية لتأمين السير العادي لمؤسسات الدولة وضمان ديمومة المرفق العمومي كما كان الحال خلال هذه السنة.\n\nمن جهة أخرى، أشار المسؤول الحكومي، ضمن جواب عن سؤال برلماني تقدم به الاتحاد الوطني للشغل بالمغرب حول “مستويات التقارب بين توقعات قوانين الميزانية وحقائق قوانين التصفية المتعلقة بها”، إلى أن توقعات قانون المالية المتعلقة بالموارد يتم تحديدها بناء على مجموعة من الفرضيات ترتبط بالظرفية الاقتصادية والمالية المتوفرة أثناء إعداد مشروع قانون المالية؛ وهو ما يفسر تسجيل بعض الفوارق بين التوقعات والإنجازات

In [ ]:
df_articles['Clean_article'][0]

'كشف فوزي لقجع، الوزير المنتدب لدى وزير الاقتصاد والمالية المكلف بالميزانية، أن الحكومة لجأت، خلال السنة الحالية، إلى فتح اعتمادات إضافية بمرسوم، بلغت 28 مليار درهم لمواجهة تداعيات تقلب أسعار بعض المواد الأولية وارتفاع معدل التضخم بفعل الحرب في أوكرانيا ويأتي هذا الإجراء في إطار ما يتمتع به الجهاز التنفيذي من صلاحيات لإجراء تغييرات على مستوى نفقات ميزانية الدولة المصادق عليها أثناء تنفيذ قانون المالية، وفق شروط وسياق يؤطرها القانون التنظيمي لقانون المالية؛ وذلك في إطار تمكينه من الوسائل الضرورية لتأمين السير العادي لمؤسسات الدولة وضمان ديمومة المرفق العمومي كما كان الحال خلال هذه السنة من جهة أخرى، أشار المسؤول الحكومي، ضمن جواب عن سؤال برلماني تقدم به الاتحاد الوطني للشغل بالمغرب حول “مستويات التقارب بين توقعات قوانين الميزانية وحقائق قوانين التصفية المتعلقة بها”، إلى أن توقعات قانون المالية المتعلقة بالموارد يتم تحديدها بناء على مجموعة من الفرضيات ترتبط بالظرفية الاقتصادية والمالية المتوفرة أثناء إعداد مشروع قانون المالية؛ وهو ما يفسر تسجيل بعض الفوارق بين التوقعات والإنجازات بفعل ال

In [ ]:
df_comm.head()

,Foreign_Key,auteurComm,Comm,Number of reaction,Clean_comment
0,0,Stop,للكرة وللحباب ولصحاب لملايير كاينة وملي الشعب ...,4,للكرة وللحباب ولصحاب لملايير كاينة وملي الشعب ...
1,0,redos,تبارك الله اللهم بارك هاذ لقجع فينما مشيتي تلق...,2,تبارك الله اللهم بارك هاذ لقجع فينما مشيتي تلق...
2,0,Diego,هل يمكن اتخاد قرارات مثل التي اتخذتها اسبانيا ...,3,هل يمكن اتخاد قرارات مثل التي اتخذتها اسبانيا ...
3,1,محمد الطربلوسي,كذلك فعل المخلوع زين العابدين بن علي،ناداه الش...,9,كذلك فعل المخلوع زين العابدين بن علي،ناداه الش...
4,1,ولد حميدو,من حقه ان يتجاهل فهل الشعب هو من أعطاه 300 ملي...,5,من حقه ان يتجاهل فهل الشعب هو من أعطاه 300 ملي...


### Downloadin datasets to csv

In [ ]:
df_articles.to_csv(r'article_info_cleaned.csv', index=False, header=True)

In [ ]:
df_comm.to_csv(r'comments_cleaned.csv', index=False, header=True)

# **2. Storage des données dans une base de données relationnel**

Pour cette partie, il faut connecter colab au runtime local, ou bien l'executer sur jupyter localement.

In [ ]:
# import packages
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# establish connections

try:
    conn = psycopg2.connect(
          user = "postgres",
          password = "postgres",
          host = "localhost",
          port = "5432",
          database = "hespress"
    )
    cur = conn.cursor()

    # Afficher la version de PostgreSQL 
    cur.execute("SELECT version();")
    version = cur.fetchone()
    print("Version : ", version,"\n")


except (Exception, psycopg2.Error) as error :
    print ("Erreur lors de la connexion à PostgreSQL", error)

# drop table if it already exists
cur.execute('drop table if exists Comments')
cur.execute('drop table if exists article')

sql = '''CREATE TABLE article(Pkey integer PRIMARY KEY,
 Titre  VARCHAR(255) ,Lien  VARCHAR(255),article text);'''
sql2 = '''CREATE TABLE Comments(PkeyComm SERIAL PRIMARY KEY,Fkey int ,
AuteurComm VARCHAR(255) ,Comm text,NumberReaction int);'''


cur.execute(sql)
cur.execute(sql2)



In [ ]:

sql = '''ALTER TABLE Comments
ADD CONSTRAINT fk_orders_customers
FOREIGN KEY (Fkey)
REFERENCES article(Pkey);'''

cur.execute(sql)
conn.commit()



Version :  ('PostgreSQL 15.1, compiled by Visual C++ build 1914, 64-bit',) 



In [ ]:

for index, row in df2.iterrows():
    cur.execute("INSERT INTO article (Pkey,Titre, Lien, article) VALUES(%s,%s,%s,%s)", (row['Primary_Key'],row['Clean_Title'], row['Article Link'], row['Clean_article']))


conn.commit()




In [ ]:


for index, row in df.iterrows():

    cur.execute("INSERT INTO Comments (Fkey,AuteurComm, Comm, NumberReaction) VALUES(%s,%s,%s,%s)", (row['Foreign_Key'],row['auteurComm'], row['Clean_comment'], row['Number of reaction']))


conn.commit()





In [ ]:

cur.execute("SELECT Titre,AuteurComm FROM article,Comments where Fkey=pkey")

# Fetch all the rows from the SELECT statement
rows = cur.fetchall()

# Print the rows
for row in rows:
    print(row)



conn.commit()




('جدل تغطية الأسود يصل إلى البرلمان والتلفزة حوادث البث عالمية', 'التاغي')
('جدل تغطية الأسود يصل إلى البرلمان والتلفزة حوادث البث عالمية', 'karim')
('جدل تغطية الأسود يصل إلى البرلمان والتلفزة حوادث البث عالمية', 'حق ي قة')
('جدل تغطية الأسود يصل إلى البرلمان والتلفزة حوادث البث عالمية', 'مواطن فخور')
('جدل تغطية الأسود يصل إلى البرلمان والتلفزة حوادث البث عالمية', 'مواطن مغربي')
('جدل تغطية الأسود يصل إلى البرلمان والتلفزة حوادث البث عالمية', 'From California Mustapha')
('جدل تغطية الأسود يصل إلى البرلمان والتلفزة حوادث البث عالمية', 'متتبع')
('جدل تغطية الأسود يصل إلى البرلمان والتلفزة حوادث البث عالمية', 'اللباس العصري')
('جدل تغطية الأسود يصل إلى البرلمان والتلفزة حوادث البث عالمية', 'المغربي المعروف')
('جدل تغطية الأسود يصل إلى البرلمان والتلفزة حوادث البث عالمية', 'abderrahmane')
('جدل تغطية الأسود يصل إلى البرلمان والتلفزة حوادث البث عالمية', 'كاسر')
('حموشي يوجه المسؤولين المركزيين والجهويين إلى تنفيذ أولويات الأمن الوطني', 'الريفي')
('حموشي يوجه المسؤولين المركزيين والجهويين 

# **3. Détection de langue des commentaires**

In [ ]:
!pip install gcld3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gcld3

detector = gcld3.NNetLanguageIdentifier(min_num_bytes=0, 
                                        max_num_bytes=1000)
text = "If you are going through hell, keep going."
result = detector.FindLanguage(text=text)
print(result.language)

text2 = "لا يوجد طريق مختصر إلى مكان يستحق الذهاب إليه"
result2 = detector.FindLanguage(text=text2)
print(result2.language)

text3 = "La nuit porte conseil"
result3 = detector.FindLanguage(text=text3)
print(result3.language)

en
ar
fr


In [ ]:
# copy the comment dataset
df_comm_copy = df_comm.copy()

#language detection of comments
import gcld3
lang_detected=[]

for comment in df_comm_copy["Clean_comment"]:
  #print(comment)
  detector = gcld3.NNetLanguageIdentifier(min_num_bytes=0,max_num_bytes=1000)
  result = detector.FindLanguage(text=comment)
  #print(result.language)
  lang_detected.append(result.language)

# create a new columns for the detected lang
df_comm_copy["Language detected"]=lang_detected

In [ ]:
df_comm_copy

,Foreign_Key,auteurComm,Comm,Number of reaction,Clean_comment,Sentiment detected,Language detected
0,0,Stop,للكرة وللحباب ولصحاب لملايير كاينة وملي الشعب ...,4,للكرة وللحباب ولصحاب لملايير كاينة وملي الشعب ...,negative,ar
1,0,redos,تبارك الله اللهم بارك هاذ لقجع فينما مشيتي تلق...,2,تبارك الله اللهم بارك هاذ لقجع فينما مشيتي تلق...,positive,ar
2,0,Diego,هل يمكن اتخاد قرارات مثل التي اتخذتها اسبانيا ...,3,هل يمكن اتخاد قرارات مثل التي اتخذتها اسبانيا ...,negative,ar
3,1,محمد الطربلوسي,كذلك فعل المخلوع زين العابدين بن علي،ناداه الش...,9,كذلك فعل المخلوع زين العابدين بن علي،ناداه الش...,negative,ar
4,1,ولد حميدو,من حقه ان يتجاهل فهل الشعب هو من أعطاه 300 ملي...,5,من حقه ان يتجاهل فهل الشعب هو من أعطاه 300 ملي...,negative,ar
...,...,...,...,...,...,...,...
1606,82,bakkalimohammed030,السلام عليكم ورحمه الله وبركاته الله يرحمنا,20,السلام عليكم ورحمه الله وبركاته الله يرحمنا,positive,ar
1607,82,ASBS RACHID VLOG,نسأل الله تعالى أن يسقنا بغيثه و يرحمنا برحمته...,20,نسأل الله تعالى أن يسقنا بغيثه و يرحمنا برحمته...,positive,ar
1608,82,ملاحظ,في التلفزيون الرسمي والغير الرسمي في الجزائر ....,7,في التلفزيون الرسمي والغير الرسمي في الجزائر و...,positive,ar
1609,82,عبد الرحيم تالوين,الحمد لله والشكر لله على نعمه التي لا تعد ولا ...,9,الحمد لله والشكر لله على نعمه التي لا تعد ولا ...,positive,ar


# **4. Detection du sentiments des commentaires**

## Détection du sentiment avec un modèle pré-entrainé

In [ ]:
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
item_counts = df_comm["Lang_detected"].value_counts()
item_counts

ar    1560
fr      44
en       4
sv       1
ur       1
sd       1
Name: Lang_detected, dtype: int64

This is a multilingual XLM-roBERTa-base model trained on ~198M tweets and finetuned for sentiment analysis. The sentiment fine-tuning was done on 8 languages (Ar, En, Fr, De, Hi, It, Sp, Pt) but it can be used for more languages (see paper for details).

In [ ]:
from transformers import pipeline
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
sentiment_task("T'estimo!")


Downloading:   0%|          | 0.00/841 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

[{'label': 'positive', 'score': 0.6600582599639893}]

In [ ]:
texts = [
    "أحب هذا الفيلم!", # "I love this movie!"
    "أكره هذا الفيلم.", # "I hate this movie."
    "لست متأكدًا من كيفية شعوري بشأن هذا الفيلم." # "I'm not sure how I feel about this movie."
]

for text in texts:
  print(f"{text}: {sentiment_task(text)}")


أحب هذا الفيلم!: [{'label': 'positive', 'score': 0.9096482396125793}]
أكره هذا الفيلم.: [{'label': 'negative', 'score': 0.9397188425064087}]
لست متأكدًا من كيفية شعوري بشأن هذا الفيلم.: [{'label': 'neutral', 'score': 0.5332158207893372}]


In [ ]:
# Test the function with a few examples
text2 = [
    "I love this movie!",
    "I hate this movie.",
    "it's natural."
]

for text in text2:
  print(f"{text}: {sentiment_task(text)}")


I love this movie!: [{'label': 'positive', 'score': 0.9369720816612244}]
I hate this movie.: [{'label': 'negative', 'score': 0.947353184223175}]
it's natural.: [{'label': 'neutral', 'score': 0.4847134053707123}]


In [ ]:
# Test the function with a few examples
text3 = [
    "Je suis très heureuse!",
    "Il est blessé.",
    "C'est la vie."
]

for text in text3:
  print(f"{text}: {sentiment_task(text)}")



Je suis très heureuse!: [{'label': 'positive', 'score': 0.9430599808692932}]
Il est blessé.: [{'label': 'negative', 'score': 0.9447647929191589}]
C'est la vie.: [{'label': 'neutral', 'score': 0.5334623456001282}]


In [ ]:
def getsentiment(text):
  s=sentiment_task(text)
  sentiment=(s[0].get('label'))
  return sentiment

In [ ]:
sentiment_detected=[]
for comment in df_comm["Comm"]:
  result =  getsentiment(comment)
  sentiment_detected.append(result)

# create a new columns for the detected lang
df_comm["Sentiment detected"]=sentiment_detected

In [ ]:
df_comm

,Foreign_Key,auteurComm,Comm,Number of reaction,Clean_comment,Sentiment detected,Lang_detected
0,0,Stop,للكرة وللحباب ولصحاب لملايير كاينة وملي الشعب ...,4,للكرة وللحباب ولصحاب لملايير كاينة وملي الشعب ...,negative,ar
1,0,redos,تبارك الله اللهم بارك هاذ لقجع فينما مشيتي تلق...,2,تبارك الله اللهم بارك هاذ لقجع فينما مشيتي تلق...,positive,ar
2,0,Diego,هل يمكن اتخاد قرارات مثل التي اتخذتها اسبانيا ...,3,هل يمكن اتخاد قرارات مثل التي اتخذتها اسبانيا ...,negative,ar
3,1,محمد الطربلوسي,كذلك فعل المخلوع زين العابدين بن علي،ناداه الش...,9,كذلك فعل المخلوع زين العابدين بن علي،ناداه الش...,negative,ar
4,1,ولد حميدو,من حقه ان يتجاهل فهل الشعب هو من أعطاه 300 ملي...,5,من حقه ان يتجاهل فهل الشعب هو من أعطاه 300 ملي...,negative,ar
...,...,...,...,...,...,...,...
1606,82,bakkalimohammed030,السلام عليكم ورحمه الله وبركاته الله يرحمنا,20,السلام عليكم ورحمه الله وبركاته الله يرحمنا,positive,ar
1607,82,ASBS RACHID VLOG,نسأل الله تعالى أن يسقنا بغيثه و يرحمنا برحمته...,20,نسأل الله تعالى أن يسقنا بغيثه و يرحمنا برحمته...,positive,ar
1608,82,ملاحظ,في التلفزيون الرسمي والغير الرسمي في الجزائر ....,7,في التلفزيون الرسمي والغير الرسمي في الجزائر و...,positive,ar
1609,82,عبد الرحيم تالوين,الحمد لله والشكر لله على نعمه التي لا تعد ولا ...,9,الحمد لله والشكر لله على نعمه التي لا تعد ولا ...,positive,ar


## Détermination du sentiment avec un modèle ML Supervisé

### Import librairies

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy, nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from xgboost.sklearn import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score
import string
seed = 100

In [ ]:
## remove stop words
!git clone https://github.com/mohataher/arabic-stop-words

Cloning into 'arabic-stop-words'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 35 (delta 0), reused 0 (delta 0), pack-reused 34
Unpacking objects: 100% (35/35), done.


In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
with open('/content/arabic-stop-words/list.txt', 'r') as f:
    stop_words = [line.strip() for line in f]
    
stop_ar_words = set(stop_words)

def remove_ar_stop_words(text):

  # Tokenize the text
  tokens = word_tokenize(text)

  # Remove the stop words
  #stop_words = set(stop_words)
  filtered_tokens = [token for token in tokens if token not in stop_ar_words]

  return filtered_tokens

In [ ]:
def clean_text(text):
  cleaned_text = remove_mult_spaces(filter_chars(strip_all_entities(strip_emoji(text))))
  article_preproced = remove_ar_stop_words(cleaned_text)
  article_preproced = " ".join(article_preproced)
  return article_preproced

### load the arabic sentiment analysis dataset

[An Arabic dataset](https://www.kaggle.com/datasets/snalyami3/arabic-sentiment-analysis-dataset-ss2030-dataset) created from Arabic Saudi tweets and is manully labelled according to the polarity that each tweet conveys (positive 1, negative 0) from kaggle


In [ ]:
import pandas as pd
train_df = pd.read_csv('arabic_sentiment_dataset.csv') 
train_df

,text,Sentiment
0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,1
1,RT @___IHAVENOIDEA: حقوق المرأة في الإسلام. ht...,1
2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,1
3,RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة ...,1
4,RT @abm112211: ولي امر الزوجة او ولي الزوجة او...,1
...,...,...
4247,#غرد_بحبك_لمحمد_بن_سلمان ❤️,1
4248,#غرد_بحبك_لمحمد_بن_سلمان \n محمدبن سلمان احبه ...,1
4249,#غرد_بحبك_لمحمد_بن_سلمان \n الله يحفظك يا ذخر ...,1
4250,#غرد_بحبك_لمحمد_بن_سلمان \n \n الله يحفظه ويحم...,1


### Build ML Classifiers

In [ ]:
#Building a pipeline
#Naive Bayes
classifier_nb = Pipeline([('tfidf',TfidfVectorizer()),
('clf',MultinomialNB())])

#Logistic regression
classifier_lr = Pipeline([('tfidf',TfidfVectorizer()),
('clf',LogisticRegression(solver='saga'))])

#Random Forest
classifier_rf = Pipeline([('tfidf',TfidfVectorizer()),
('clf',RandomForestClassifier(bootstrap= False, criterion= 'entropy', n_estimators= 100))])

#Linear SVC
classifier_svc = Pipeline([('tfidf',TfidfVectorizer()),
('clf',LinearSVC())])
#XGBoost
classifier_xgb = Pipeline([('tfidf',TfidfVectorizer()),
('clf',XGBClassifier())])

In [ ]:
clfs = [classifier_nb, classifier_lr, classifier_rf, classifier_svc, classifier_xgb]
X = train_df["text"]
y = train_df["Sentiment"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
for clf in clfs:
  clf.fit(X_train, y_train)

In [ ]:
def analyze(clf):
  #Predicting
  y_pred = clf.predict(X_test)
  yt_pred = clf.predict(X_train)
  #Analyzing
  cm = confusion_matrix(y_test,y_pred)
  print(f'Confusion Matrix :\n {cm}\n')
  test_set_acc=accuracy_score(y_test,y_pred)
  print(f'Test Set Accuracy Score :\n {accuracy_score(y_test,y_pred)}\n')
  train_set_acc=accuracy_score(y_train,yt_pred)
  print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
  classif_report=classification_report(y_test,y_pred)
  print(f'Classification Report :\n {classification_report(y_test,y_pred)}')
  return clf,cm,test_set_acc,train_set_acc,classif_report


In [ ]:
for clf in clfs:
  print(clf)
  analyze(clf)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])
Confusion Matrix :
 [[219 144]
 [  4 484]]

Test Set Accuracy Score :
 0.8260869565217391

Train Set Accuracy Score :
 0.9582475742428698

Classification Report :
               precision    recall  f1-score   support

           0       0.98      0.60      0.75       363
           1       0.77      0.99      0.87       488

    accuracy                           0.83       851
   macro avg       0.88      0.80      0.81       851
weighted avg       0.86      0.83      0.82       851

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LogisticRegression(solver='saga'))])
Confusion Matrix :
 [[258 105]
 [ 26 462]]

Test Set Accuracy Score :
 0.8460634547591069

Train Set Accuracy Score :
 0.9635401352543369

Classification Report :
               precision    recall  f1-score   support

           0       0.91      0.71      0.80       363
           1       0.81      0.95      0.88       488

 

### Predict sentiment on our comments dataset

In [ ]:
y_final = classifier_lr.predict(df_comm['Clean_comment'])
df_comm['Predicted_sentiment2']=y_final
#preds = pd.DataFrame(y_final, columns=['prediction'])
#preds


In [ ]:
df_comm

,Foreign_Key,auteurComm,Comm,Number of reaction,Clean_comment,Sentiment detected,Lang_detected,Predicted_sentiment2
0,0,Stop,للكرة وللحباب ولصحاب لملايير كاينة وملي الشعب ...,4,للكرة وللحباب ولصحاب لملايير كاينة وملي الشعب ...,negative,ar,1
1,0,redos,تبارك الله اللهم بارك هاذ لقجع فينما مشيتي تلق...,2,تبارك الله اللهم بارك هاذ لقجع فينما مشيتي تلق...,positive,ar,1
2,0,Diego,هل يمكن اتخاد قرارات مثل التي اتخذتها اسبانيا ...,3,هل يمكن اتخاد قرارات مثل التي اتخذتها اسبانيا ...,negative,ar,0
3,1,محمد الطربلوسي,كذلك فعل المخلوع زين العابدين بن علي،ناداه الش...,9,كذلك فعل المخلوع زين العابدين بن علي،ناداه الش...,negative,ar,0
4,1,ولد حميدو,من حقه ان يتجاهل فهل الشعب هو من أعطاه 300 ملي...,5,من حقه ان يتجاهل فهل الشعب هو من أعطاه 300 ملي...,negative,ar,1
...,...,...,...,...,...,...,...,...
1606,82,bakkalimohammed030,السلام عليكم ورحمه الله وبركاته الله يرحمنا,20,السلام عليكم ورحمه الله وبركاته الله يرحمنا,positive,ar,1
1607,82,ASBS RACHID VLOG,نسأل الله تعالى أن يسقنا بغيثه و يرحمنا برحمته...,20,نسأل الله تعالى أن يسقنا بغيثه و يرحمنا برحمته...,positive,ar,1
1608,82,ملاحظ,في التلفزيون الرسمي والغير الرسمي في الجزائر ....,7,في التلفزيون الرسمي والغير الرسمي في الجزائر و...,positive,ar,0
1609,82,عبد الرحيم تالوين,الحمد لله والشكر لله على نعمه التي لا تعد ولا ...,9,الحمد لله والشكر لله على نعمه التي لا تعد ولا ...,positive,ar,1


# **5. Détermination des topics des articles**

In [ ]:
# we start with installing bertopic from pypi before preparing the data
!pip install bertopic[all]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 8.1 MB/s 
     |████████████████████████████████| 662 kB 74.1 MB/s 
     |████████████████████████████████| 85 kB 7.2 MB/s 
     |████████████████████████████████| 88 kB 10.3 MB/s 
     |████████████████████████████████| 5.2 MB 63.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 5.8 MB 67.7 MB/s 
     |████████████████████████████████| 1.3 MB 71.0 MB/s 
     |████████████████████████████████| 182 kB 71.8 MB/s 
     |████████████████████████████████| 7.6 MB 66.9 MB/s 
     |████████████████████████████████| 1.1 MB 60.3 MB/s 
     |████████████████████████████████| 127 kB 74.4 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700867 sha256=59804be645286f4f0b7a2b02406b8783ff5f7fadbc

In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 15.8 MB/s 
     |████████████████████████████████| 1.2 MB 70.3 MB/s 
     |████████████████████████████████| 19.7 MB 492 kB/s 
     |████████████████████████████████| 1.6 MB 66.8 MB/s 
     |████████████████████████████████| 788 kB 75.0 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 981 kB 70.5 MB/s 
     |████████████████████████████████| 200 kB 81.0 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=1041f80f9e088bb3ad735022e8a828dd6830cbfa7ac6b9b9635ddf5ff40ffa7f
  Stored in directory: /root/.cache/pip/wheels/3d/9f/9d/d806a20bd97bc7076d724fa3e69fa5be61836ba16b2ffa6126
  Created wheel for overrides: filename=overrides-3.1.0-py3-none-any.whl size=10187 sha256=e504b3f05c541fcd47e62c26c671e5ebbef0ca

In [ ]:
# after restarting the runtime we lose local variables
# so we will redefine our dataframes from csv files installed previuosly
df_articles = pd.read_csv('article_info_cleaned.csv')
df_comm = pd.read_csv('comments_cleaned.csv')

In [ ]:
df_articles.head()

,Primary_Key,Article Link,Article_title,Article,Clean_article,Clean_Title
0,0,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تخصص 28 مليار درهم لمواجهة التضخم وتدا...,كشف فوزي لقجع، الوزير المنتدب لدى وزير الاقتصا...,كشف فوزي لقجع، الوزير المنتدب لدى وزير الاقتصا...,الحكومة تخصص 28 مليار درهم لمواجهة التضخم وتدا...
1,1,https://www.hespress.com/%d8%a7%d9%84%d8%b1%d8...,الرئيس التونسي قيس سعيّد يتجاهل المقاطعة الواس...,قلّل الرئيس التونسي قيس سعيّد، الأربعاء، من أه...,قلّل الرئيس التونسي قيس سعيّد، الأربعاء، من أه...,الرئيس التونسي قيس سعيّد يتجاهل المقاطعة الواس...
2,2,https://www.hespress.com/%d9%85%d9%8a%d8%b1%d8...,ميراوي يعترف بصعوبة تعويض الأساتذة المقبلين عل...,قال عبد اللطيف ميراوي، وزير التعليم العالي وال...,قال عبد اللطيف ميراوي، وزير التعليم العالي وال...,ميراوي يعترف بصعوبة تعويض الأساتذة المقبلين عل...
3,3,https://www.hespress.com/%d8%b1%d8%b5%d9%8a%d9...,رصيف الصحافة: أطباء يحذرون من اتساع الممارسة ا...,قراءة مواد بعض الجرائد الخاصة بيوم الخميس نسته...,قراءة مواد بعض الجرائد الخاصة بيوم الخميس نسته...,رصيف الصحافة أطباء يحذرون من اتساع الممارسة ال...
4,4,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تنجح في إقناع مهندسي الملاحة الجوية ال...,تراجع مهندسو وأطر سلامة الملاحة الجوية بمطارات...,تراجع مهندسو وأطر سلامة الملاحة الجوية بمطارات...,الحكومة تنجح في إقناع مهندسي الملاحة الجوية ال...


In [ ]:
df_articles['Clean_article'][0]

'كشف فوزي لقجع، الوزير المنتدب لدى وزير الاقتصاد والمالية المكلف بالميزانية، أن الحكومة لجأت، خلال السنة الحالية، إلى فتح اعتمادات إضافية بمرسوم، بلغت 28 مليار درهم لمواجهة تداعيات تقلب أسعار بعض المواد الأولية وارتفاع معدل التضخم بفعل الحرب في أوكرانيا ويأتي هذا الإجراء في إطار ما يتمتع به الجهاز التنفيذي من صلاحيات لإجراء تغييرات على مستوى نفقات ميزانية الدولة المصادق عليها أثناء تنفيذ قانون المالية، وفق شروط وسياق يؤطرها القانون التنظيمي لقانون المالية؛ وذلك في إطار تمكينه من الوسائل الضرورية لتأمين السير العادي لمؤسسات الدولة وضمان ديمومة المرفق العمومي كما كان الحال خلال هذه السنة من جهة أخرى، أشار المسؤول الحكومي، ضمن جواب عن سؤال برلماني تقدم به الاتحاد الوطني للشغل بالمغرب حول “مستويات التقارب بين توقعات قوانين الميزانية وحقائق قوانين التصفية المتعلقة بها”، إلى أن توقعات قانون المالية المتعلقة بالموارد يتم تحديدها بناء على مجموعة من الفرضيات ترتبط بالظرفية الاقتصادية والمالية المتوفرة أثناء إعداد مشروع قانون المالية؛ وهو ما يفسر تسجيل بعض الفوارق بين التوقعات والإنجازات بفعل ال

In [ ]:
from bertopic import BERTopic
from flair.embeddings import TransformerDocumentEmbeddings
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
from gensim.models import LdaMulticore
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [ ]:
#to experiment with other BERT models simply change the model name below
arabert = TransformerDocumentEmbeddings('aubmindlab/bert-base-arabertv02')

Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/825k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [ ]:
topic_model = BERTopic(language="arabic", low_memory=True ,calculate_probabilities=False,
                     embedding_model=arabert)

### Removing Stop words

In [ ]:
articles_preproceesed= []
for t in df_articles['Clean_article']:
    articles_preproceesed.append(remove_ar_stop_words(t))

In [ ]:
articles_preproceesed[0]

['كشف',
 'فوزي',
 'لقجع،',
 'الوزير',
 'المنتدب',
 'وزير',
 'الاقتصاد',
 'والمالية',
 'المكلف',
 'بالميزانية،',
 'الحكومة',
 'لجأت،',
 'السنة',
 'الحالية،',
 'فتح',
 'اعتمادات',
 'إضافية',
 'بمرسوم،',
 'بلغت',
 '28',
 'لمواجهة',
 'تداعيات',
 'تقلب',
 'أسعار',
 'المواد',
 'الأولية',
 'وارتفاع',
 'معدل',
 'التضخم',
 'بفعل',
 'الحرب',
 'أوكرانيا',
 'ويأتي',
 'الإجراء',
 'إطار',
 'يتمتع',
 'الجهاز',
 'التنفيذي',
 'صلاحيات',
 'لإجراء',
 'تغييرات',
 'مستوى',
 'نفقات',
 'ميزانية',
 'الدولة',
 'المصادق',
 'أثناء',
 'تنفيذ',
 'قانون',
 'المالية،',
 'وفق',
 'شروط',
 'وسياق',
 'يؤطرها',
 'القانون',
 'التنظيمي',
 'لقانون',
 'المالية؛',
 'وذلك',
 'إطار',
 'تمكينه',
 'الوسائل',
 'الضرورية',
 'لتأمين',
 'السير',
 'العادي',
 'لمؤسسات',
 'الدولة',
 'وضمان',
 'ديمومة',
 'المرفق',
 'العمومي',
 'الحال',
 'السنة',
 'جهة',
 'أخرى،',
 'أشار',
 'المسؤول',
 'الحكومي،',
 'جواب',
 'سؤال',
 'برلماني',
 'تقدم',
 'الاتحاد',
 'الوطني',
 'للشغل',
 'بالمغرب',
 '“',
 'مستويات',
 'التقارب',
 'توقعات',
 'قوانين',
 'الميز

In [ ]:
articles_preproceesed = [' '.join(sublist) for sublist in articles_preproceesed]
df_articles['RmvStpWrds_article'] = articles_preproceesed
df_articles['RmvStpWrds_article'][0]

'كشف فوزي لقجع، الوزير المنتدب وزير الاقتصاد والمالية المكلف بالميزانية، الحكومة لجأت، السنة الحالية، فتح اعتمادات إضافية بمرسوم، بلغت 28 لمواجهة تداعيات تقلب أسعار المواد الأولية وارتفاع معدل التضخم بفعل الحرب أوكرانيا ويأتي الإجراء إطار يتمتع الجهاز التنفيذي صلاحيات لإجراء تغييرات مستوى نفقات ميزانية الدولة المصادق أثناء تنفيذ قانون المالية، وفق شروط وسياق يؤطرها القانون التنظيمي لقانون المالية؛ وذلك إطار تمكينه الوسائل الضرورية لتأمين السير العادي لمؤسسات الدولة وضمان ديمومة المرفق العمومي الحال السنة جهة أخرى، أشار المسؤول الحكومي، جواب سؤال برلماني تقدم الاتحاد الوطني للشغل بالمغرب “ مستويات التقارب توقعات قوانين الميزانية وحقائق قوانين التصفية المتعلقة ” توقعات قانون المالية المتعلقة بالموارد يتم تحديدها بناء مجموعة الفرضيات ترتبط بالظرفية الاقتصادية والمالية المتوفرة أثناء إعداد مشروع قانون المالية؛ يفسر تسجيل الفوارق التوقعات والإنجازات بفعل التغييرات الظرفية تنفيذ قانون المالية، بالإضافة المداخيل الجبائية تتميز بطابعها الطارئ وغير الاعتيادي؛ وبالتالي فهي تدخل التوقعات السنوية 

In [ ]:
topics, probs = topic_model.fit_transform(df_articles['RmvStpWrds_article'])

In [ ]:
#extract most frequent topics
topic_model.get_topic_freq()

,Topic,Count
0,0,36
1,-1,26
2,1,21


In [ ]:
#show the top 10 words in topic 0
topic_model.get_topic(0)

[('الكحول', 0.025457316310702936),
 ('القصر', 0.02263243123788066),
 ('الكبير', 0.02154313621901898),
 ('فريق', 0.018385839557729895),
 ('المغربي', 0.018081656581737614),
 ('مدينة', 0.016889585377910084),
 ('المهاجرين', 0.016155255578870514),
 ('القدم', 0.015420925779830948),
 ('المغرب', 0.015183640959284561),
 ('لكرة', 0.015057782938798371)]

In [ ]:
#show the top 10 words in topic 1
topic_model.get_topic(1)

[('المغرب', 0.020300712490480623),
 ('القضائية', 0.019968879139077012),
 ('العدل', 0.018335985137827995),
 ('اللغة', 0.018335985137827995),
 ('السلطة', 0.017743343948153988),
 ('العالم', 0.01565630447223751),
 ('القانون', 0.01564723926565437),
 ('التعليم', 0.0141458524133767),
 ('المجلس', 0.013996437105410705),
 ('وبالتالي', 0.01384320925663291)]

In [ ]:
#show the top 10 words in topic -1 (outlier to ignore)
topic_model.get_topic(-1)

[('المغربية', 0.02231263116792817),
 ('النقل', 0.016681574589053776),
 ('الحدود', 0.01636951921725257),
 ('فيما', 0.0161010456448491),
 ('المغربي', 0.01604785488562131),
 ('المغاربة', 0.014939034227178535),
 ('المائة', 0.014133295657063723),
 ('الوطني', 0.014090414343957152),
 ('الهندي', 0.013327533835783395),
 ('جعيط', 0.013327533835783395)]

In [ ]:
topic_model.visualize_barchart()

In [ ]:
len(topics)

83

In [ ]:
df_articles['Article_Topic']=topics

In [ ]:
#df_articles.drop(['Article_Topic'], axis = 1, inplace = True)
#df_articles.rename(columns={'Article_Topic2': 'Article_Topic'},inplace=True, errors='raise')

In [ ]:
df_articles

,Primary_Key,Article Link,Article_title,Article,Clean_article,Clean_Title,RmvStpWrds_article,Article_Topic
0,0,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تخصص 28 مليار درهم لمواجهة التضخم وتدا...,كشف فوزي لقجع، الوزير المنتدب لدى وزير الاقتصا...,كشف فوزي لقجع، الوزير المنتدب لدى وزير الاقتصا...,الحكومة تخصص 28 مليار درهم لمواجهة التضخم وتدا...,كشف فوزي لقجع، الوزير المنتدب وزير الاقتصاد وا...,1
1,1,https://www.hespress.com/%d8%a7%d9%84%d8%b1%d8...,الرئيس التونسي قيس سعيّد يتجاهل المقاطعة الواس...,قلّل الرئيس التونسي قيس سعيّد، الأربعاء، من أه...,قلّل الرئيس التونسي قيس سعيّد، الأربعاء، من أه...,الرئيس التونسي قيس سعيّد يتجاهل المقاطعة الواس...,قلّل الرئيس التونسي قيس سعيّد، الأربعاء، أهمية...,-1
2,2,https://www.hespress.com/%d9%85%d9%8a%d8%b1%d8...,ميراوي يعترف بصعوبة تعويض الأساتذة المقبلين عل...,قال عبد اللطيف ميراوي، وزير التعليم العالي وال...,قال عبد اللطيف ميراوي، وزير التعليم العالي وال...,ميراوي يعترف بصعوبة تعويض الأساتذة المقبلين عل...,عبد اللطيف ميراوي، وزير التعليم العالي والبحث ...,1
3,3,https://www.hespress.com/%d8%b1%d8%b5%d9%8a%d9...,رصيف الصحافة: أطباء يحذرون من اتساع الممارسة ا...,قراءة مواد بعض الجرائد الخاصة بيوم الخميس نسته...,قراءة مواد بعض الجرائد الخاصة بيوم الخميس نسته...,رصيف الصحافة أطباء يحذرون من اتساع الممارسة ال...,قراءة مواد الجرائد الخاصة بيوم الخميس نستهلها ...,1
4,4,https://www.hespress.com/%d8%a7%d9%84%d8%ad%d9...,الحكومة تنجح في إقناع مهندسي الملاحة الجوية ال...,تراجع مهندسو وأطر سلامة الملاحة الجوية بمطارات...,تراجع مهندسو وأطر سلامة الملاحة الجوية بمطارات...,الحكومة تنجح في إقناع مهندسي الملاحة الجوية ال...,تراجع مهندسو وأطر سلامة الملاحة الجوية بمطارات...,0
...,...,...,...,...,...,...,...,...
78,78,https://www.hespress.com/%d8%af%d8%b1%d9%88%d8...,"دروس وعبر من ""مونديال"" قطر!",انتهت كما هو معلوم منافسات كأس العالم في نسخته...,انتهت كما هو معلوم منافسات كأس العالم في نسخته...,دروس وعبر من مونديال قطر,انتهت معلوم منافسات كأس العالم نسختها 22، حظيت...,-1
79,79,https://www.hespress.com/%d8%ad%d9%8a%d9%86-%d...,حين عاد الأبطال الصناديد إلى أرض الوطن,بعد ملحمة الدوحة، عاد أبناء الوطن الى حضن الوط...,بعد ملحمة الدوحة، عاد أبناء الوطن الى حضن الوط...,حين عاد الأبطال الصناديد إلى أرض الوطن,ملحمة الدوحة، أبناء الوطن حضن الوطن، فوجدوا ان...,0
80,80,https://www.hespress.com/%d8%a7%d9%84%d8%aa%d8...,التعليم.. إصلاح قد لا يأتي,المتأمل لكرونولوجيا ما يسمى “إصلاح التعليم” با...,المتأمل لكرونولوجيا ما يسمى “إصلاح التعليم” با...,التعليم إصلاح قد لا يأتي,المتأمل لكرونولوجيا يسمى “ إصلاح التعليم ” بال...,1
81,81,https://www.hespress.com/%d8%a7%d9%84%d9%88%d9...,الوكالات الجهوية للتعمير في خدمة الاسثثمار,راكمت الوكالات الحضرية، منذ إحداثها خلال عقد ا...,راكمت الوكالات الحضرية، منذ إحداثها خلال عقد ا...,الوكالات الجهوية للتعمير في خدمة الاسثثمار,راكمت الوكالات الحضرية، إحداثها عقد الثمانينات...,1


## **6. Visualisation des résultats en PowerBI**

# **Dowload datasets resuls**

In [ ]:
df_articles.to_csv(r'article_sentiment_analysis.csv', index=False, header=True)
df_comm.to_csv(r'comments_sentiment_analysis.csv', index=False, header=True)